In [1]:
import bs4
from IPython.display import HTML

from gensim import summarization
from pyMorfologik import Morfologik
from pyMorfologik.parsing import ListParser

import editdistance
import flashtext

In [2]:
note_html = """\ufeff<p>Cluster</p>\n<ul>\n<li>\n<p>nodes</p>\n</li>\n<li>\n<p>pods</p>\n<ul>\n<li>pod is a group of containers with shared storage and networking (IP address)e</li>\n<li>pody są najmniejszymi jednostkami na których stawiane są kontenery</li>\n<li>ilość podów kontroluje parametr replica</li>\n</ul>\n</li>\n<li>\n<p>deployments</p>\n<ul>\n<li>deploymenty są obiektami które kontrolują stan opisany w konfiguracji, np odpal 3 kontenery/pody serwera Apache. Odpowiadają one za skalowanie (odpalanie odpowiedniej ilości kontenerów podów i utrzymywanie jej, np zwiększanie gdy zużycie CPU przekracza na jednym pewien próg)</li>\n</ul>\n</li>\n<li>\n<p>ingress</p>\n</li>\n</ul>\n<p>Networking</p>\n<ul>\n<li>pod dostaje jeden adres IP (adresy są ephemeral)</li>\n<li>pody na jednym node są w jednym networking namespace (widzą swoje prywatne IP?)</li>\n<li>Node dostają adresy IP z adres range z VPC</li>\n</ul>\n<p>Service</p>\n<ul>\n<li>services give pods IP address and name</li>\n<li>mają statyczne wirtualne adresy IP</li>\n<li>są 3 typy: Cluster IP, node port, load balancer (każdy następny rozbudowany nad poprzednim)\n<ul>\n<li>Cluser IP przypisuje stałe IP wewnątrz klastra (do komunikacji podów)</li>\n<li>Node port służy do komunikacji na zewnątrz klastra</li>\n<li>Load balancer jest node portem który dodatkowo pomaga rozkładać ruch</li>\n</ul>\n</li>\n<li>serwisy są przypisane do applikacji</li>\n</ul>\n<p>Ingress</p>\n<ul>\n<li>'a service for services'</li>\n<li>ingress resource exposes services  (pole backend)</li>\n<li>może łączyć się z NodePortami lub Load Balancerami</li>\n</ul>\n<p>StatefulSet\nStatefulSet jest jakby Deploymentem który zachowuje stan (w każdym podzie)\nPody są domyślnie odpalane sekwencyjnie (0, 1 itd) ale mogą być odpalane równolegle</p>\n<p>Volumes</p>\n<ul>\n<li>share state/data between pods</li>\n<li>Ephemeral\n<ul>\n<li>per pod</li>\n<li>emptyDir - ephemeral</li>\n<li>ConfigMap, Secret, downwardAPI</li>\n</ul>\n</li>\n<li>PersistentDisk\n<ul>\n<li>per klaster</li>\n<li>są tworzone odrębnie od wykorzystania (provision vs claim) - muszą być stworzone przed używaniem</li>\n</ul>\n</li>\n</ul>\n"""

In [3]:
note_soup = bs4.BeautifulSoup(note_html)

/home/kuba/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /home/kuba/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [4]:
def get_edit_close_words(w, checked_words, max_dist):
    return [word for word in checked_words if editdistance.eval(w, word) <= max_dist]

def replace_edit_close_words(text, keywords, max_dist):
    words = text.split()
    replaced_words = set().union(*(get_edit_close_words(w, words, max_dist=2) for w in keywords))
    keyword_processor = flashtext.KeywordProcessor()
    for word in replaced_words:
        keyword_processor.add_keyword(word, '****')
    return keyword_processor.replace_keywords(text)


def replace_keywords_in_paragraph(html_paragraph, max_dist):
    text = html_paragraph.text
    keywords = summarization.keywords(text, 0.1, split=True)    
    return replace_edit_close_words(text, keywords, max_dist)


def replace_keywords(html_text, max_dist=2):
    paragraphs = html_text.findAll('ul')
    replaced_paragraphs = [replace_keywords_in_paragraph(paragraph, max_dist) for paragraph in paragraphs]
    return '\n'.join(replaced_paragraphs)

In [5]:
print(replace_keywords(note_soup))



nodes


****

**** is a group of containers with shared storage and networking (IP address)e
**** są najmniejszymi jednostkami na których stawiane są ****
ilość **** kontroluje parametr replica



deployments

deploymenty są obiektami które kontrolują stan opisany w konfiguracji, np odpal 3 ****/**** serwera Apache. Odpowiadają one za skalowanie (odpalanie odpowiedniej ilości **** **** i utrzymywanie jej, np zwiększanie gdy zużycie CPU przekracza na jednym **** próg)



ingress



pod is a group of containers with shared storage and networking (IP address)e
pody są najmniejszymi jednostkami na których stawiane są kontenery
ilość podów kontroluje **** replica


deploymenty są obiektami **** kontrolują stan opisany w konfiguracji, np odpal 3 kontenery/pody serwera Apache. Odpowiadają one za skalowanie (odpalanie odpowiedniej ilości kontenerów podów i utrzymywanie jej, np zwiększanie gdy zużycie CPU przekracza na jednym **** próg)


pod **** jeden adres IP (adresy są ephemeral)
pody na 